In [4]:
!pip install sklearn.ensemble
!pip install sklearn.metrics
!pip install pyarrow
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement sklearn.ensemble (from versions: none)
ERROR: No matching distribution found for sklearn.ensemble
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement sklearn.metrics (from versions: none)
ERROR: No matching distribution found for sklearn.metrics
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 5.5 MB/s eta 0:00:00:00:0100:01
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.1 MB/s eta 0:00:00:00:0100:01


In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

train_path = "../data/training_data.parquet"

train_df = pd.read_parquet(train_path)
print("Training data shape: ", train_df.shape)

# # Train-test split
# X = df.iloc[:, :-1]
# y = df.iloc[:, [-1]]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
# train_data = pd.concat([X_train, y_train], axis = 1)
# test_data = pd.concat([X_test, y_test], axis = 1)

ArrowKeyError: No type extension with name arrow.py_extension_type found